# Great Circle Distance and Bearings
Geodesy using spherical trigonometry

In [1]:
# Basic definitions v0.4

from math import sin, asin, cos, acos, atan, pi, radians, degrees, sqrt
from spherical import DMS, Spherical, Geo
a = DMS()

Most geodesy is done using classic degree, minute, second, direction designations (e.g. 83&deg; 15' 22" W) but the math requires floating point (real) decimal numbers

The trig functions are provided as methods of classes to provide visibility to the internal values

The laws of spherical geometry are different than those of planar geometry

1) Let ABC be a spherical triangle with angle A of 30 degrees, B of 15 degrees and side length a of 45 degrees. Calculate side length b.

In [2]:
Spherical().law_of_sine_ASA(30,45,15)

21.470701432439956

2) Let DEF be a spherical triangle with angle F of 60 degrees and side lengths d and e of 75 and 30 degrees respectively. Calculate side length f and angle D.

In [3]:
f = Spherical().law_of_cosine(75,30,60)
f

62.24930050756424

In [4]:
Spherical().law_of_sine_SSA(f, 75, 60)

70.95023417122484

What is the solid angle of the Sun?
What percentage of the sky is occupied by the Sun?
The distance to the Sun is r = 149.6 * 10^6 km
The diameter of the Sun is D = 1.392 * 10^6 km

In [5]:
r = 149.6e6
D = 1.392e6
theta = atan((D/2)/r)
angle = DMS(degrees(theta))
domega = Spherical.delta_omega(angle.dd)
f"theta: {angle} delta omega: {domega:.2g} -> {(domega * r * 100) / (4*pi*r):.2g}%"

'theta: 0° 15\' 59.62" (0.26656°) delta omega: 6.8e-05 -> 0.00054%'

In [6]:
Geo("Lisbon", DMS().assign(38, 43, 31), 'N', DMS().assign(9, 9, 0), 'W').trip(
    Geo("Moscow", DMS().assign(55, 45, 21), 'N', DMS().assign(37, 37, 2), 'E'))
# solution says: 3906km (5h 25m)

From Lisbon to Moscow: 3906.3km (5.43hrs) start: 45° 26' 11.81" (45.43662°) finish: 98° 58' 08.45" (98.96902°)


In [7]:
Geo("Tokyo", DMS().assign(35, 41, 23), 'N', DMS().assign(139, 41, 32), 'E').trip(
    Geo("London", DMS().assign(51, 30, 26), 'N', DMS().assign(0, 7, 39), 'W'))
# solution says: 13.3 hrs = 47850s

From Tokyo to London: 9558.7km (13.28hrs) start: -23° 44' 22.01" (-23.73945°) finish: 211° 41' 23.87" (211.68996°)


In [8]:
Geo("Rio de Janeiro", DMS().assign(22, 54, 40), 'S', DMS().assign(43, 12, 20), 'W').trip(
    Geo("Beijing", DMS().assign(39, 54, 24), 'N', DMS().assign(116, 23, 51), 'E'))
# Rio 22°54′40″S 43°12′20″W
# Beijing 39°54′24″N 116°23′51″E

From Rio de Janeiro to Beijing: 17322.7km (24.06hrs) start: 40° 40' 54.15" (40.68171°) finish: 128° 29' 16.78" (128.48799°)


In [9]:
madrid = Geo("Madrid", DMS().assign(40, 23, 0), 'N', DMS().assign(3, 43, 0), 'W')
tokyo = Geo("Tokyo", DMS().assign(35, 41, 0), 'N', DMS().assign(139, 41, 0), 'E')
madrid.trip(tokyo)
print(f"path: {DMS(madrid.gc_path)} upper latitude: {DMS(madrid.upper_lat)}")
# Madrid 40°25′01″N 03°42′12″W

From Madrid to Tokyo: 10766.2km (14.95hrs) start: 29° 11' 31.55" (29.19210°) finish: 152° 46' 50.10" (152.78058°)
path: 96° 49' 20.80" (96.82245°) upper latitude: 68° 11' 25.29" (68.19036°)


In [10]:
cairo = Geo("Cairo", DMS().assign(30, 2, 40), 'N', DMS().assign(31, 14, 9), 'E')
sfo = Geo("San Francisco", DMS().assign(37, 46, 39), 'N', DMS().assign(122, 24, 59), 'W')
cairo.trip(sfo)
print(f"path: {DMS(cairo.gc_path)} upper latitude: {DMS(cairo.upper_lat)}")

From Cairo to San Francisco: 11991.6km (16.65hrs) start: -21° 37' 27.31" (-21.62425°) finish: 203° 48' 12.14" (203.80337°)
path: 107° 50' 34.76" (107.84299°) upper latitude: 71° 23' 50.49" (71.39736°)


In [11]:
Geo("Caledonia", DMS().assign(40, 37, 54.6), 'N', DMS().assign(82, 58, 36), 'W').trip(
    Geo("Jerusalem", DMS().assign(31, 46, 8.54), 'N', DMS().assign(35, 12, 59) , 'E'))

From Caledonia to Jerusalem: 9765.2km (13.56hrs) start: 48° 34' 40.24" (48.57784°) finish: 137° 58' 57.07" (137.98252°)


In [12]:
Geo("Jerusalem", DMS().assign(31, 46, 8.54), 'N', DMS().assign(35, 12, 59) , 'E').trip(Geo("Caledonia", DMS().assign(40, 37, 54.6), 'N', DMS().assign(82, 58, 36), 'W'))

From Jerusalem to Caledonia: 9765.2km (13.56hrs) start: -42° 1' 02.93" (-42.01748°) finish: 228° 34' 40.24" (228.57784°)


In [16]:
from pyproj import geod
import numpy as np
azis = np.ndarray(shape=(5,))
print(f'overall {geod.Geod(ellps="WGS84").inv(cairo.longi, cairo.lat, sfo.longi, sfo.lat)}')
course = geod.Geod(ellps="WGS84").inv_intermediate(cairo.longi, cairo.lat, sfo.longi, sfo.lat, npts=5, out_azis=azis)
for lat, lon, azm in zip(course.lats, course.lons, azis):
    print(f"lat: {lat} lon: {lon} azm: {azm}")

overall (-21.585917849285355, 23.750175461710512, 12014070.076677624)
lat: 46.4964579952737 lon: 21.75368372571081 azm: -27.528416898559247
lat: 61.45411576472998 lon: 4.420985077342621 azm: -41.70308139673779
lat: 71.02605091387552 lon: -34.63040285160365 azm: -77.7844085242814
lat: 67.05491879433372 lon: -85.1696967710154 azm: -125.38741374990065
lat: 53.73873784950124 lon: -110.30448581316598 azm: -147.47219284255448


In [15]:
f"{sin(radians(360000030)):.10f}"

'0.5000000001'